# Web Scraping del sito Subito.it



## Import delle librerie


In [21]:
import requests
import bs4
import csv
from tqdm import tqdm_notebook as tqdm
import json
import pprint
import re

## Definizione url di riferimento

Per semplificare il progetto la ricerca riguarderà tutti gli annunci trovati filtrando per regione Lombardia e prodotto iPhone.


In [2]:
webpage = f"https://www.subito.it/annunci-lombardia/vendita/usato/?q=iPhone&o=1"

Cerco tutti gli annunci nella pagina:

In [3]:
response = requests.get(webpage)
doc = bs4.BeautifulSoup(response.text)
annunci = doc.find_all('a', class_="AdElements__Item--link-L2hvbWUv")

In [4]:
len(annunci)

33

Ci sono 33 annunci nella pagina!

Test del codice per lo scarico delle prime informazioni nella pagina di ricerca degli annunci:

In [5]:
annunci_list = []
for annuncio in annunci:
    titolo = annuncio.find("h2").text.replace("\n", "").strip()
    annuncio_id = annuncio["href"]
    url = annuncio["href"]
    try:
        prezzo = annuncio.find("div",class_ = "AdElements__ItemPrice--container-L2hvbWUv").text.replace("\n", "").replace("€","").strip()
    except:
        prezzo = "N/A"
    try:
        data_paese = annuncio.select(" div.AdElements__Item--dateLocation-L2hvbWUv > div > span")[0].text
    except:
        data_paese = "N/A"
    annunci_list.append({'id': annuncio_id, 'titolo': titolo, 'link': url, 
                         'prezzo': prezzo, 'datapaese': data_paese})
    
print(annunci_list[0])

{'id': 'https://www.subito.it/telefonia/iphone-se-64gb-milano-300445718.htm', 'titolo': 'IPhone SE 64GB', 'link': 'https://www.subito.it/telefonia/iphone-se-64gb-milano-300445718.htm', 'prezzo': '150', 'datapaese': 'Oggi, 15:39 - Milano (MI)'}


Ok, ora posso definire la funzione per il parsing generico degli annunci:

In [6]:

def parse_annuncio(annuncio):
    titolo = annuncio.find("h2").text.replace("\n", "").strip()
    url = annuncio["href"]
    annuncio_id = url.split('-')[-1].split('.')[0]
    try:
        prezzo = annuncio.find("div",class_ = "AdElements__ItemPrice--container-L2hvbWUv").text.replace("\n", "").replace("€","").strip()
    except:
        prezzo = "N/A"
    try:
        data_paese = annuncio.select(" div.AdElements__Item--dateLocation-L2hvbWUv > div > span")[0].text
    except:
        data_paese = "N/A"
    
    return {'id': annuncio_id, 'titolo': titolo, 'link': url, 
                         'prezzo': prezzo, 'datapaese': data_paese}


In un ciclo for gestisco il download degli annunci per ogni pagina:

In [7]:
annunci_list = []
for num in tqdm(range(1,11)):
    webpage = f"https://www.subito.it/annunci-lombardia/vendita/usato/?q=iPhone&o={num}"
    response = requests.get(webpage)
    doc = bs4.BeautifulSoup(response.text)
    annunci = doc.find_all('a', class_="AdElements__Item--link-L2hvbWUv")
    for annuncio in annunci:
        annunci_list.append(parse_annuncio(annuncio))

print(len(annunci_list))
        


330


## Come terminare lo scraping?

Problema: *quando mi fermo?*
- cerco all'interno della pagina il numero di annuncio totali e continuo finco a che il numero degli annunci letti non è uguale al totale


In [8]:
firstpage = f"https://www.subito.it/annunci-lombardia/vendita/usato/?q=iPhone&o=1"
firstresponse = requests.get(firstpage)
firstdoc = bs4.BeautifulSoup(firstresponse.text)
adscount = int(firstdoc.find_all("div", class_="ads-count")[0].select("span")[0].text.replace('.',''))
print(adscount)

5502


In [39]:
annunci_list = []
annunci_letti = 0
num = 0
while annunci_letti < adscount:
    num = num + 1
    webpage = f"https://www.subito.it/annunci-lombardia/vendita/usato/?q=iPhone&o={num}"
    response = requests.get(webpage)
    doc = bs4.BeautifulSoup(response.text)
    annunci = doc.find_all('a', class_="AdElements__Item--link-L2hvbWUv")
    for annuncio in annunci:
        try:
            annunci_list.append(parse_annuncio(annuncio))
        except:
            pass
    annunci_letti = len(annunci_list)

KeyboardInterrupt: 

## Pandas
Utilizzo la libreria Pandas per salvare i risultati in un file csv:

In [9]:
import pandas as pd
ds_annunci = pd.DataFrame(annunci_list)
ds_annunci.set_index("id")
ds_annunci.head()

,datapaese,id,link,prezzo,titolo
0,"Oggi, 15:39 - Milano (MI)",300445718,https://www.subito.it/telefonia/iphone-se-64gb...,150,IPhone SE 64GB
1,"Oggi, 15:33 - Monza (MB)",300445119,https://www.subito.it/telefonia/iphone-xr-azzu...,669,Iphone xr azzurro 128gb garanzia apple
2,"Oggi, 15:31 - Magenta (MI)",300444918,https://www.subito.it/telefonia/iphone-xs-max-...,N/A,IPhone XS Max
3,"Oggi, 15:30 - Saronno (VA)",300444839,https://www.subito.it/telefonia/iphone-5s-16gb...,90,IPHONE 5S 16gb
4,"Oggi, 15:24 - Monza (MB)",300444118,https://www.subito.it/telefonia/xr-product-red...,669,Xr product red 128gb garanzia apple


In [10]:
ds_annunci.to_csv("./subito_it_annunci.csv")

## Le pagine degli annunci ###
Ora l'obiettivo è scaricare i dettagli dei singoli annunci e le foto.
# Geocoding
Per avere una maggiore precisione riguardo all'informazione geografica degli annunci, richiamo il servizio di geocoding in modo da ottenere una geolocalizzazione più strutturata.


In [18]:
import json
# apro il file csv
import pandas as pd
ds_annunci = pd.read_csv("./subito_it_annunci.csv", index_col="id")
ds_annunci.head()

,Unnamed: 0,datapaese,link,prezzo,titolo
id,,,,,
300445718,0,"Oggi, 15:39 - Milano (MI)",https://www.subito.it/telefonia/iphone-se-64gb...,150.0,IPhone SE 64GB
300445119,1,"Oggi, 15:33 - Monza (MB)",https://www.subito.it/telefonia/iphone-xr-azzu...,669.0,Iphone xr azzurro 128gb garanzia apple
300444918,2,"Oggi, 15:31 - Magenta (MI)",https://www.subito.it/telefonia/iphone-xs-max-...,NaN,IPhone XS Max
300444839,3,"Oggi, 15:30 - Saronno (VA)",https://www.subito.it/telefonia/iphone-5s-16gb...,90.0,IPHONE 5S 16gb
300444118,4,"Oggi, 15:24 - Monza (MB)",https://www.subito.it/telefonia/xr-product-red...,669.0,Xr product red 128gb garanzia apple


Per ogni annuncio vado a richiamare il link e a procedere con lo scarico dalla pagina di dettaglio:

In [20]:
dettagli = []
for annuncio_id, annuncio in tqdm(ds_annunci.iterrows(), total=ds_annunci.shape[0]):
    dettagli_row = {}
    link = annuncio["link"]
    response = requests.get(link)
    doc = bs4.BeautifulSoup(response.text)
    try:
        descrizione = doc.find("div", class_="description").text.replace('\n', ' ').strip()
        regex = "([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)"
        p = re.findall(regex, descrizione)
        if len(p) > 0:
            email = p[0] # prendo la prima per semplicità
        else:
            email = ''
    except:
        continue
    try:
        nome_venditore = doc.select("div.publisher_info > h2")[0].text.replace('\n', ' ').strip()
    except:
        nome_venditore = "N/A"
    try: 
        phone = doc.select("div#contact-actions-container")[0]["data-prop-phone"]
    except:
        phone = 'N/A'
    try: 
        userId = doc.select("div#user_info_container")[0]["data-user-id"]
    except:
        userId = 'N/A'
    id_ad = doc.select("main")[0]["data-id"]
    date_ad = doc.select("time")[0]["datetime"]
    dettagli_row.update({'id': id_ad, 'descrizione': descrizione,  'nome_venditore': nome_venditore,
                        'phone': phone, 'date_ad': date_ad, 'userId':userId, 'email' : email})
    table = doc.select("div.summary > table ")[0]
    new_table = pd.DataFrame(columns=range(0,2), index = [0]) 
    row_marker = 0
    for row in table.find_all('tr'):
        columns = row.find_all('td')
        label = columns[0].text.replace('\n', ' ').strip()
        value = columns[1].text.replace('\n', ' ').strip()
        dettagli_row.update({label:value})
    ## geocoding
    lat = ""
    lon = ""
    try:
        key = "T4eqDjtnpzWsfeMBZgKAqKobvcICurpU"
        geocode_url = f"http://www.mapquestapi.com/geocoding/v1/address?key={key}&location={dettagli_row.get('Comune').split(' ')[0]}"
        response = requests.get(geocode_url)
        geo = json.loads(response.text)
        lat = geo['results'][0]['locations'][0]['latLng']['lat']
        lon = geo['results'][0]['locations'][0]['latLng']['lng']
    except:
        pass
    dettagli_row.update({"lat": lat, "lon": lon})
    #scarico le immagini
    imgs = doc.select("img")
    i = 0
    for img in imgs:
        src = img['src']
        if src.find("logo_print.png") > 0:
            # E' il logo, non lo salvo
            print('E il logo')
        else:
            img_file = requests.get(src, stream=True)
            if img_file.status_code == 200:
                with open("/home/master/carla/progetto_bigdata/img/img_" + str(id_ad) + "_" + str(i) + ".jpg", 'wb') as f:
                    f.write(img_file.content)
                i = i+1
    dettagli.append(dettagli_row)

ConnectionError: HTTPSConnectionPool(host='www.subito.it', port=443): Max retries exceeded with url: /telefonia/spina-apple-originale-per-alimentatore-nuova-milano-300341000.htm (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x7f49a0fe51d0>: Failed to establish a new connection: [Errno 113] No route to host',))

In [13]:
ds_dettagli = pd.DataFrame(dettagli)
ds_dettagli.set_index("id")
ds_dettagli.head()

,Allestimento,Anno immatricolazione,Cambio,Carburante,Classe emissioni,Comune,Km,Locali,Marca,Modello,...,Tipologia,Titolo di Studio,date_ad,descrizione,id,lat,lon,nome_venditore,phone,userId
0,NaN,NaN,NaN,NaN,NaN,Milano (MI),NaN,NaN,NaN,NaN,...,Cellulari e Smartphone,NaN,2019-06-15 15:39:07,Vendo iPhone SE 64GB gold rose condizioni prat...,300445718,45.4668,9.1905,Madda,N/A,100923714
1,NaN,NaN,NaN,NaN,NaN,Monza (MB),NaN,NaN,NaN,NaN,...,Cellulari e Smartphone,NaN,2019-06-15 15:33:15,INTERPLANET VIALE ROMAGNA 39 CINISELLO BALSAMO...,300445119,45.5834,9.27353,N/A,3395619991,N/A
2,NaN,NaN,NaN,NaN,NaN,Magenta (MI),NaN,NaN,NaN,NaN,...,Cellulari e Smartphone,NaN,2019-06-15 15:31:33,Scambio iPhone XS Max 256 giga ( ripeto ..256 ...,300444918,49.0481,3.96651,iPhone XS Max256,N/A,715243
3,NaN,NaN,NaN,NaN,NaN,Saronno (VA),NaN,NaN,NaN,NaN,...,Cellulari e Smartphone,NaN,2019-06-15 15:30:41,Vendo Iphone 5s usato con pochissimi segni di ...,300444839,45.6279,9.03464,Aydin,3298180429,19128195
4,NaN,NaN,NaN,NaN,NaN,Monza (MB),NaN,NaN,NaN,NaN,...,Cellulari e Smartphone,NaN,2019-06-15 15:24:38,INTERPLANET VIALE ROMAGNA 39 CINISELLO BALSAMO...,300444118,45.5834,9.27353,N/A,3395619991,N/A


In [14]:
ds_dettagli.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330 entries, 0 to 329
Data columns (total 29 columns):
Allestimento             3 non-null object
Anno immatricolazione    4 non-null object
Cambio                   4 non-null object
Carburante               4 non-null object
Classe emissioni         3 non-null object
Comune                   321 non-null object
Km                       4 non-null object
Locali                   1 non-null object
Marca                    4 non-null object
Modello                  4 non-null object
Orario                   9 non-null object
Partita IVA              9 non-null object
Porte                    4 non-null object
Posti                    4 non-null object
Prezzo                   309 non-null object
Sede di lavoro           9 non-null object
Settore                  9 non-null object
Superficie               1 non-null object
Tipo di sport            1 non-null object
Tipologia                303 non-null object
Titolo di Studio         9 no

In [15]:
ds_dettagli.to_csv("./subito_it_dettagli.csv")